In [ ]:
# Tell the notebook where to find the pipeline functions and scripts
import sys
sys.path.append('/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/Lesion_Pipeline/')

In [ ]:
import os
import shutil
from glob import glob 
import GNMpipeline as p

# Add your inputs

In [ ]:
input_dir="/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/Pipeline/RDCRN_test_output/"  #Folder with input files, can use the folder created by Combine script previously
input_type='BIDS' #'BIDS' or 'Folder'
output_dir="/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/Pipeline/RDCRN_test_output/"

subjects_to_skip=None

skullstrip='both' #synthstrip, optibet, both, None 
                        

IMAGE_TYPES = ['t1', 'T2', 'FLAIR'] #case sensitive, change to match what you used e.g. t1, t1w, TI 
Reg_target_1='t1' #your ideal registration target, case sensitive
Reg_target_2='T2'

# Make dictionary of subjects and sessions

In [ ]:
subject_sessions=p.create_input_dict(input_dir, subjects_to_skip=subjects_to_skip, input_type=input_type)

# Prepare your registration image with bias correcting etc.

In [ ]:
bias_corr_dict={}
for subject, sessions in subject_sessions.items():
    reg_target=None
    subject_folder = os.path.join(output_dir, subject) 
    for session in sessions:
        print(f'*** Processing {subject}: {session} ***')
        session_folder=os.path.join(subject_folder, session)
        
        reg_target=p.set_registration_target(glob(f'{session_folder}/SELECTED/*.nii*'), Reg_target_1, Reg_target_2 )
        print(reg_target)
        
        reg_image=f'{session_folder}/SELECTED/{subject}_{session}_{reg_target}.nii.gz'
        
        print(f'Reslicing {reg_image}')
        #reslice_image(reg_image)
        
        print(f'Bias Correcting {reg_image}')
        if '1' in reg_target:
            img_type='T1'
        elif '2' in reg_target:
            img_type='T2'
        
        command=p.bias_corr(reg_image, img_type, skullstrip=skullstrip, clean_up=True) 
        job_name = f"bias_correct_{subject}_{session}_{reg_target}"
        job_id=p.submit_slurm_job(job_name, command)
        bias_corr_dict[(subject,session)] = job_id

if len(bias_corr_dict) > 0:
    print('You have', len(bias_corr_dict), 'Bias Correction Jobs submitted to SLURM; subject and job IDs are stored in bias_corr_dict')
    print('You can type "squeue -u $USER" into your terminal to track SLURM job progress')
    print('You can check the output file matching the jobid in bias_corr_dict to see code outputs and any errors')
else:
    print('You have', len(bias_corr_dict), 'Bias Correction Jobs')

# Check your submitted jobs

In [ ]:
print("These are the jobs that should have been submitted:")
print(bias_corr_dict)

In [ ]:
!squeue -u $USER 